## Segmenting and Clustering Neighborhooods in Toronto
## Part 2: Add coordinates to every postal code

### Applied Data Science Project

#### Luis Andrade, August 2021

To achieve this tasks we need Pandas and Geocoder libraries

In [ ]:
!pip install geocoder # Run if geocoder library is not installed

In [1]:
import pandas as pd
import geocoder

We will import the postal code dataframe that we previously built on the previous excercise. We also drop unnecesary colums

In [2]:
df = pd.read_csv('toronto_codes.csv')
df.drop(['Unnamed: 0'], axis=1, inplace=True)
print(df.shape)
df.head(10)

(103, 3)


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


We are going to inlude coordinates to every postal code sample. We have two options. The first one will be using **geocoder** library. The seconnd one is using a coordinate table.

## Option 1: Geocoder library

We iterate for every postal code to get their reference coordinates through the geocoder library. However, this package has reported problems to succesfully retrieve them. Instead, it many times yields null results. For that reason we included a `while` loop that keeps trying getting coordinates while the results are null.

In [ ]:
lat_long_coords = []

for postcode in df['PostalCode']:
    lat_long = None
    while (lat_long is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postcode))
        lat_long = g.latlng
    lat_long_coords.append(lat_long)

Unfortunately this path didn't give us the expected results. Had it worked, we would have populated the postal code table with new latitude and longitude attributes.

In [ ]:
df['Latitude'] = lat_long_coords[0]
df['Longitude'] = lat_long_coords[1]
df.head()

### Option 2: Geospatial data table

Should option 1 fail we then try the geospatial table alternative. Firstly we import the csv file from our repository.

The code in the cell below comes from Watson Studio's `Find and add data` options `Insert to code` followed by `pandas DataFrame`

In [3]:
# The code was removed by Watson Studio for sharing.

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [3]:
# RUN IF USING LOCAL JUPYTER NOTEBOOK
#coordinates_df = pd.read_csv('Geospatial_Coordinates.csv')
#coordinates_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


What we are going to do is to merge this coordinates to the original postal code data frame. But first we are going to rename the postal code column of the coordinates table to that name of the postal code table, from 'Postal Code' to 'PostalCode', in order to perform a table merge, using as reference this attribute.

In [4]:
coordinates_df.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
print(coordinates_df.shape)
coordinates_df.head(3)

(103, 3)


,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711


Next, we integrate these coordinates to the original postal codes table. We will perform a table merge using as reference the postal codes and givinng priority to the postal code table, acting as a left argument. This is equivalent to an SQL left outer join, that in the `pd.merge` function is reppresented by the argument `how='left'`. 

Alternatively, since in this case both tables have the same samples i.e. postal codes, we could also perform an inner join, a table intersection, without worrying about loosing postal code samples on the original table. We do this by setting the argument `how='inner'` in the `pd.merge` function.

In [5]:
df = pd.merge(df, coordinates_df, how='left', on='PostalCode')
print(df.shape)
df.head(10)

(103, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


Now we have an updated table with the previous attributes (postal code, borough and neighborhood) plus the coordinates (latitude and longitude), five atributes for the 103 samples.

Finally, we could export this table to a csv file.

In [6]:
df.to_csv('toronto_codes_coord.csv')